en los apartados donde pone audio/audio2.mp3, substituir por la carpeta que hayas creado y el nombre del archibo de audio que haya dentro

In [7]:
!pip install SpeechRecognition pydub
!pip install fpdf
from fpdf import FPDF
from pydub import AudioSegment
from pydub.utils import which
import speech_recognition as sr
import os
import re

In [8]:
audio_path = "audio.mp3"

In [9]:
AudioSegment.converter = which("ffmpeg")

# Ruta del archivo de audio (cámbiala por la tuya)
audio_path = "audio/Audio2.mp3"

audio_wav = "audio_temp.wav"
AudioSegment.from_file(audio_path).export(audio_wav, format="wav")

<_io.BufferedRandom name='audio_temp.wav'>

In [10]:
# (cámbiala por la tuya)
audio_path = "audio/Audio2.mp3"


audio_wav = "audio_temp.wav"
AudioSegment.from_file(audio_path).export(audio_wav, format="wav")


recognizer = sr.Recognizer()


with sr.AudioFile(audio_wav) as source:
    print("Cargando el audio...")
    audio_data = recognizer.record(source)

try:
    transcription = recognizer.recognize_google(audio_data, language="es-ES")
    print("Transcripción:")
    print(transcription)
except sr.UnknownValueError:
    print("No se pudo entender el audio.")
except sr.RequestError as e:
    print(f"Error del servicio de reconocimiento: {e}")


Cargando el audio...
Error del servicio de reconocimiento: recognition connection failed: [Errno 32] Broken pipe


In [11]:
output_folder = "segmentos"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

audio = AudioSegment.from_file("audio_temp.wav")

segment_duration = 60 * 1000
for i, segment in enumerate(audio[::segment_duration]):
    segment_path = os.path.join(output_folder, f"segment_{i}.wav")
    segment.export(segment_path, format="wav")
    print(f"Segmento {i} exportado a: {segment_path}")

Segmento 0 exportado a: segmentos/segment_0.wav
Segmento 1 exportado a: segmentos/segment_1.wav
Segmento 2 exportado a: segmentos/segment_2.wav
Segmento 3 exportado a: segmentos/segment_3.wav
Segmento 4 exportado a: segmentos/segment_4.wav
Segmento 5 exportado a: segmentos/segment_5.wav
Segmento 6 exportado a: segmentos/segment_6.wav
Segmento 7 exportado a: segmentos/segment_7.wav
Segmento 8 exportado a: segmentos/segment_8.wav
Segmento 9 exportado a: segmentos/segment_9.wav
Segmento 10 exportado a: segmentos/segment_10.wav
Segmento 11 exportado a: segmentos/segment_11.wav
Segmento 12 exportado a: segmentos/segment_12.wav
Segmento 13 exportado a: segmentos/segment_13.wav


In [12]:
audio_directory = "segmentos"
audio_files = [file for file in os.listdir(audio_directory) if file.endswith(".wav")]


def sort_key(file_name):
    match = re.search(r"segment_(\d+)", file_name)
    return int(match.group(1)) if match else float('inf')

audio_files = sorted(audio_files, key=sort_key)

print("Archivos de audio ordenados:", audio_files)

texto_completo = ""

for audio_file in audio_files:
    audio_path = os.path.join(audio_directory, audio_file)
    print(f"Procesando: {audio_file}")

    try:
        with sr.AudioFile(audio_path) as source:
            audio_data = recognizer.record(source)
            transcription = recognizer.recognize_google(audio_data, language="es-ES")
            print(f"Transcripción de {audio_file}: {transcription}\n")
            texto_completo += transcription + " "
    except sr.UnknownValueError:
        print(f"No se pudo entender el audio: {audio_file}")
        texto_completo += "[No se pudo entender este fragmento.] "
    except sr.RequestError as e:
        print(f"Error del servicio de reconocimiento: {e}")
        texto_completo += f"[Error del servicio: {e}] "

output_file = "transcripcion_final.txt"
with open(output_file, "w", encoding="utf-8") as file:
    file.write(texto_completo)
print(f"Transcripción guardada en {output_file}")

Archivos de audio ordenados: ['segment_0.wav', 'segment_1.wav', 'segment_2.wav', 'segment_3.wav', 'segment_4.wav', 'segment_5.wav', 'segment_6.wav', 'segment_7.wav', 'segment_8.wav', 'segment_9.wav', 'segment_10.wav', 'segment_11.wav', 'segment_12.wav', 'segment_13.wav']
Procesando: segment_0.wav
Transcripción de segment_0.wav: una comunicación entre Cali y Windows vale Si no tenemos la red Nat porque podemos tener varias redes más montadas para nuestros equipos por ejemplo en casa por wi-fi no es lo mismo que cuando lo hago por cable os habéis dado cuenta que cuando voy por cable me entregan $122 168.2 y tal en cambio por wi-fi me da otro otro Rango pero normalmente empiezan por 10 entonces podéis tener varias redes Nat sí se entiende la idea Este es el primer problema porque si no no llegaré a destino y no podré analizar y podré ver lo que pasa en la máquina objetivo que en este caso la máquina objetivo estaría preparada dentro de lo que es Cali de acuerdo Entonces qué he hecho prime

In [15]:
input_txt_file = "transcripcion_final.txt"
output_pdf_file = "transcripcion_final.pdf"

pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)
pdf.add_page()
pdf.set_font("Arial", size=12)

with open(input_txt_file, "r", encoding="utf-8") as file:
    for line in file:
        pdf.multi_cell(0, 10, line.encode('latin-1', 'replace').decode('latin-1'))

pdf.output(output_pdf_file)
print(f"Archivo PDF guardado como {output_pdf_file}")

Archivo PDF guardado como transcripcion_final.pdf
